### compare the number of qtls found with PC and PEERs residualization

look at https://www.biorxiv.org/content/10.1101/2022.03.09.483661v1.full.pdf for examples of how to do this type of comparision

* number of discovered cis-eGenes
* number of credible sets

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import spearmanr, linregress
import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')


from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

In [2]:
test = {}
len(test)

0